In [1]:
import os

In [2]:
%pwd

'/Users/raj.muniraju/Documents/Workspace/my_ml_projects/myFirstMLDeployment/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/raj.muniraju/Documents/Workspace/my_ml_projects/myFirstMLDeployment'

In [22]:
# os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/tmbharathiraja/End_to_end_mlops.mlflow"
# os.environ["MLFLOW_TRACKING_USERNAME"]="tmbharathiraja"
# os.environ["MLFLOW_TRACKING_PASSWORD"]="Open$ource@123"

import dagshub
dagshub.init(repo_owner='tmbharathiraja', repo_name='myFirstMLDeployment', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/Users/raj.muniraju/Documents/Workspace/my_ml_projects/myFirstMLDeployment/env/lib/python3.8/site-packages/rich/liv
e.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=cf4bc247-ef4f-4b4b-aff9-18a981297bf1&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=0c8218ce4acf67b17d086a9d8487e7ba76c6e63ed18664e4571997adbab55dde


[2025-01-26 14:12:07,433: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2025-01-26 14:12:07,570: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2025-01-26 14:12:07,919: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as tmbharathiraja

[2025-01-26 14:12:07,923: INFO: helpers: Accessing as tmbharathiraja]
[2025-01-26 14:12:08,266: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/tmbharathiraja/myFirstMLDeployment "HTTP/1.1 200 OK"]
[2025-01-26 14:12:08,622: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "tmbharathiraja/myFirstMLDeployment"

[2025-01-26 14:12:08,625: INFO: helpers: Initialized MLflow to track repo "tmbharathiraja/myFirstMLDeployment"]


Repository tmbharathiraja/myFirstMLDeployment initialized!

[2025-01-26 14:12:08,626: INFO: helpers: Repository tmbharathiraja/myFirstMLDeployment initialized!]


In [29]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [30]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            # mlflow_uri="https://dagshub.com/someshnaman/End_to_end_MLOPS_project.mlflow",
            mlflow_uri="https://dagshub.com/tmbharathiraja/myFirstMLDeployment.mlflow",
           
        )

        return model_evaluation_config


In [32]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [33]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        test_data = test_data.drop(columns=["type"])
        test_data = test_data.dropna()
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [34]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-26 14:17:52,025: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-26 14:17:52,029: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-26 14:17:52,033: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-26 14:17:52,037: INFO: common: created directory at: artifacts]
[2025-01-26 14:17:52,041: INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-26 14:17:52,245: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/01/26 14:17:58 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.


In [ ]:
print("ashok Jallepalli")

ashok Jallepalli
